In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV 
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline 
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier 
from xgboost import XGBClassifier 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score 
import joblib

In [8]:
df = pd.read_csv('telco_churn.csv')

In [9]:
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


## Data Cleaning

In [22]:
# ========================= # Data Cleaning / Preprocessing # ========================= 
nis_cols = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies'] 
df[nis_cols] = df[nis_cols].replace('No internet service', 'No') 
df['MultipleLines'] = df['MultipleLines'].replace('No phone service', 'No') 

yes_no_cols = ['Churn', 'PaperlessBilling', 'StreamingMovies', 'StreamingTV', 'TechSupport', 'MultipleLines', 'DeviceProtection', 'OnlineBackup', 'OnlineSecurity', 'PhoneService', 'Dependents', 'Partner'] 

# Convert Yes/No → 1/0 
df[yes_no_cols] = df[yes_no_cols].replace({'Yes':1, 'No':0}).astype(int) 

# Convert numeric columns 
num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges'] 
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce') 
df['TotalCharges'] = df['TotalCharges'].fillna(df['tenure']*df['MonthlyCharges']) 

# Identify categorical columns for OneHotEncoding 
cat_cols = df.select_dtypes(include=['object']).columns.tolist() 

# ========================= # Split data # ========================= 
X = df.drop('Churn', axis=1) 
y = df['Churn'] 

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, stratify=y, random_state=42 )


## Model Comparison

In [23]:
# ========================= # Preprocessor & Pipeline # ========================= 
preprocessor = ColumnTransformer(transformers=[ 
    ('num', StandardScaler(), num_cols), 
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
]) 

# ========================= # Model Comparison # ========================= 
def compare_model(model_class): 
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) 
    
    pipeline = Pipeline([ ('preprocess', preprocessor), 
                          ('model', model_class()) 
                        ]) 
    
    score = cross_val_score(pipeline, X_train, y_train, cv=skf, scoring='recall').mean() 
    
    return score 
    
results = { 'Logistic Regression': compare_model(LogisticRegression), 
            'Decision Tree': compare_model(DecisionTreeClassifier), 
            'Random Forest': compare_model(RandomForestClassifier), 
            'XGBoost': compare_model(XGBClassifier) } 
    
df_results = pd.DataFrame.from_dict(results, orient='index', columns=['Recall (CV - Train)']).sort_values(by='Recall (CV - Train)', ascending=False) 
print(df_results)

                     Recall (CV - Train)
Logistic Regression             0.521070
XGBoost                         0.506355
Decision Tree                   0.488294
Random Forest                   0.485619


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib

## Data Cleaning

In [4]:
# =========================
# Data Cleaning / Preprocessing
# =========================
nis_cols = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
df[nis_cols] = df[nis_cols].replace('No internet service', 'No')
df['MultipleLines'] = df['MultipleLines'].replace('No phone service', 'No')

yes_no_cols = ['Churn', 'PaperlessBilling', 'StreamingMovies', 'StreamingTV', 'TechSupport',
               'MultipleLines', 'DeviceProtection', 'OnlineBackup', 'OnlineSecurity', 
               'PhoneService', 'Dependents', 'Partner']

# Convert Yes/No → 1/0
df[yes_no_cols] = df[yes_no_cols].replace({'Yes':1, 'No':0}).astype(int)

# Convert numeric columns
num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(df['tenure']*df['MonthlyCharges'])

# Identify categorical columns for OneHotEncoding
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

# =========================
# Split data
# =========================
X = df.drop('Churn', axis=1)
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

C:\Users\felip\AppData\Local\Temp\ipykernel_11260\3437131033.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[yes_no_cols] = df[yes_no_cols].replace({'Yes':1, 'No':0}).astype(int)


## Model Comparison

In [5]:
# =========================
# Preprocessor & Pipeline
# =========================
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

# =========================
# Model Comparison
# =========================
def compare_model(model_class):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    pipeline = Pipeline([
        ('preprocess', preprocessor),
        ('model', model_class())
    ])
    score = cross_val_score(pipeline, X_train, y_train, cv=skf, scoring='recall').mean()
    return score

results = {
    'Logistic Regression': compare_model(LogisticRegression),
    'Decision Tree': compare_model(DecisionTreeClassifier),
    'Random Forest': compare_model(RandomForestClassifier),
    'XGBoost': compare_model(XGBClassifier)
}

df_results = pd.DataFrame.from_dict(results, orient='index', columns=['Recall (CV - Train)']).sort_values(by='Recall (CV - Train)', ascending=False)
print(df_results)


                     Recall (CV - Train)
Logistic Regression             0.521070
XGBoost                         0.506355
Decision Tree                   0.493645
Random Forest                   0.489632


## Hyperparameter tuning and metrics comparison

In [24]:
# =========================
# Hyperparameter Tuning
# =========================
pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', LogisticRegression(max_iter=1000))
])

param_grid = {
    'model__C': [0.01, 0.1, 1, 10],
    'model__solver': ['lbfgs', 'liblinear'],
    'model__penalty': ['l2'],
    'model__class_weight': [None, 'balanced']
}

grid = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring='recall', cv=5, n_jobs=-1)
grid.fit(X_train, y_train)

y_pred = grid.best_estimator_.predict(X_test)
recall_final = recall_score(y_test, y_pred)
print("Recall after tuning:", recall_final)

# =========================
# Metrics Comparison
# =========================
baseline_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', LogisticRegression(max_iter=1000))
])
baseline_pipeline.fit(X_train, y_train)
y_pred_baseline = baseline_pipeline.predict(X_test)

optimized_pipeline = grid.best_estimator_
y_pred_optimized = optimized_pipeline.predict(X_test)

baseline_metrics = {
    'Accuracy': accuracy_score(y_test, y_pred_baseline),
    'Precision': precision_score(y_test, y_pred_baseline),
    'Recall': recall_score(y_test, y_pred_baseline),
    'F1-score': f1_score(y_test, y_pred_baseline)
}

optimized_metrics = {
    'Accuracy': accuracy_score(y_test, y_pred_optimized),
    'Precision': precision_score(y_test, y_pred_optimized),
    'Recall': recall_score(y_test, y_pred_optimized),
    'F1-score': f1_score(y_test, y_pred_optimized)
}

df_comparison = pd.DataFrame([baseline_metrics, optimized_metrics], index=['Baseline', 'Optimized']).T
print(df_comparison)



C:\Users\felip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


Recall after tuning: 0.7994652406417112
           Baseline  Optimized
Accuracy   0.794890   0.737402
Precision  0.636656   0.503367
Recall     0.529412   0.799465
F1-score   0.578102   0.617769


## Save model

In [17]:
# =========================
# Save model & columns
# =========================
joblib.dump(grid.best_estimator_, 'churn_model.pkl')
model_columns = X_train.columns.tolist()
joblib.dump(model_columns, 'model_columns.pkl')

['model_columns.pkl']